MARKDOWN

Importing necessary libraries

In [27]:
import matplotlib.pyplot as plt
import pandas as pd
import math
import numpy as np

Custom Functions

In [60]:
# define calculation of concrete volume
def calculate_concrete_volume(diameter, length):
    concrete_volume = length * (diameter ** 2) * math.pi / 4
    return concrete_volume

# function for selecting the appropriate steel diameter
def round_number(steel_diameter, steel_list):
    steel_diameter_fixed = round(steel_diameter)

    if steel_diameter_fixed in steel_list:
        return steel_diameter_fixed

    steel_fixed = min(steel_list, key=lambda x: abs(x - steel_diameter_fixed))
    return steel_fixed

# define calculation of steel volume
def calculate_steel_volume(bending_moment, diameter, length, yield_strength_steel, steel_list):
    steel_area = bending_moment * 1000000 / (0.87 * diameter * 1000 * yield_strength_steel)
    steel_diameter = pow(steel_area * 4 / (6 * math.pi), 0.5)
    steel_diameter_fixed = round_number(steel_diameter, steel_list)
    steel_area_fixed = 6 * (steel_diameter_fixed ** 2) * math.pi / (4 * 1000000)
    steel_volume = steel_area_fixed * length
    return steel_volume

#define carbon emission and cost
def calculate_concrete_carbon_cost(concrete_volume, material_density, carbon_factor, cost_factor):
    carbon_emission = concrete_volume * material_density * carbon_factor
    cost = concrete_volume * material_density * cost_factor 
    concrete_carbon_cost = carbon_emission + cost
    return concrete_carbon_cost

def calculate_steel_carbon_cost(steel_volume):
    steel_density = 1500 #kg per cubic
    carbon_factor = 1383.31
    cost_factor = 500
    s_carbon_emission = steel_volume * steel_density * carbon_factor
    s_cost = steel_volume * steel_density * cost_factor
    steel_carbon_cost = s_carbon_emission + s_cost
    return steel_carbon_cost


Carbon Cost Material Dictionaries

In [61]:
# Carbon factors for different materials and energy (in kilograms of CO2 per unit)
carbon_factors = {
    'CEM I': 860,
    'Aggregate': 4,
    'sand': 6,
    'water': 0.149,
    'bentonite': 539,
}

# Cost factors for different materials (in pounds of cost per unit)
cost_factors = {
    'CEM I': 200,
    'Aggregate': 10,
    'sand': 10,
    'water': 10,
    'bentonite': 150,
}

# density for different materials (in kg per cubic)
material_densities = {
    'CEM I': 400,
    'Aggregate':900,
    'Sand': 800,
    'Water': 100,
    'bentonite':600,
}

Importing data and manual parameter specification

In [45]:
#input data from outside, example in excel here
df = pd.read_excel('Book1.xlsx')
print(df.columns)
length = 26 # variable of embedment length
yield_strength_steel = 500 / 1.15 # partial factor for the strength of steel
steel_list = [8, 10, 12, 16, 20, 25, 32, 40, 50]

Index(['A', 'B'], dtype='object')


Calculations for steel and concrete volume

In [53]:
bending_moment = df['A'] # Maximum bending moment of the wall
diameter = df['B'] # Diameter of the columns

concrete_volume = []
for i in range(len(diameter)):   
    result = calculate_concrete_volume(diameter[i], length)
    concrete_volume.append (result)

steel_volume = []
for i in range(len(bending_moment)):
    result = calculate_steel_volume(bending_moment[i], diameter[i], length, yield_strength_steel, steel_list)
    steel_volume.append (result)
    
#create dataframe for the result
output_df = pd.DataFrame({'concrete volume': concrete_volume, 'steel volume': steel_volume})
output_df


,concrete volume,steel volume
0,29.405307,0.049009
1,20.420352,0.049009
2,13.069025,0.031366
3,10.005973,0.031366
4,7.351327,0.031366
5,5.105088,0.031366
6,3.267256,0.031366


In [69]:
total_carbon_cost = 0.0

# Make data into array
concrete_volume = np.array(concrete_volume)
steel_volume = np.array(steel_volume)

# Calculate carbon cost for each material
for material, carbon_factor in carbon_factors.items():
    carbon_factor = carbon_factors.get(material, 0)
    cost_factor = cost_factors.get(material, 0)
    material_density = material_densities.get(material,0)
    
concrete_carbon_cost = calculate_concrete_carbon_cost(concrete_volume, material_density, carbon_factor, cost_factor)
steel_carbon_cost = calculate_steel_carbon_cost(steel_volume)
total_carbon_cost = (concrete_carbon_cost + steel_carbon_cost)

# Create DataFrame for visualization
output_df2 = pd.DataFrame({'concrete': concrete_carbon_cost,'steel': steel_carbon_cost})
output_df2 


,concrete,steel
0,1.215615e+07,138448.272934
1,8.441774e+06,138448.272934
2,5.402735e+06,88606.894678
3,4.136469e+06,88606.894678
4,3.039039e+06,88606.894678
5,2.110443e+06,88606.894678
6,1.350684e+06,88606.894678
